# 트랜스포머(Transformer)를 GPT-1 모델로 변경

이 노트북은 기존의 인코더-디코더(Encoder-Decoder) 구조의 트랜스포머 모델을 GPT-1 논문("Improving Language Understanding by Generative Pre-Training")의 핵심 아이디어를 적용하여 **디코더-온리(Decoder-Only)** 구조로 변경하는 과정을 담고 있습니다.

### 주요 구조 변경 사항 및 근거

| 변경 사항 | 내용 | 근거 |
| :--- | :--- | :--- |
| **인코더(Encoder) 블록 제거** | 입력 문장을 인코딩하는 인코더 스택 전체를 제거합니다. | GPT는 번역과 같은 Seq2Seq 과업이 아닌, 주어진 컨텍스트를 바탕으로 다음에 올 단어를 예측하는 **언어 모델(Language Model)**입니다. 따라서 입력과 출력을 분리하여 처리하는 인코더-디코더 구조가 불필요하며, 단일 디코더 블록만으로 구성됩니다. |
| **디코더(Decoder) 블록 수정** | 디코더 레이어 내부에 존재하던 **Encoder-Decoder Attention** 부분을 제거합니다. | 인코더가 제거되었으므로, 인코더의 출력 결과를 참조하는 Encoder-Decoder Attention 또한 필요하지 않습니다. GPT의 디코더 블록은 오직 **Masked Self-Attention**과 **Feed-Forward Network** 두 개의 서브-레이어로만 구성됩니다. |
| **데이터 전처리 방식 변경** | 질문(Q)과 답변(A)을 별도로 처리하지 않고, `질문 + 구분자 + 답변` 형태의 단일 시퀀스로 통합합니다. | GPT는 **자기회귀(Autoregressive)** 방식으로 이전 단어들을 바탕으로 다음 단어를 예측하며 학습합니다. 질문을 컨텍스트로, 답변을 생성해야 할 텍스트로 인식시키기 위해 두 문장을 자연스럽게 연결한 단일 데이터로 만들어 모델이 문맥을 학습하고 답변을 생성하도록 유도합니다. |
| **입력 데이터 구성** | 모델의 최종 입력은 **토큰 임베딩**과 **포지셔널 임베딩(위치 정보)**의 합으로 구성됩니다. | GPT는 RNN과 달리 단어의 순서를 직접 학습할 수 없으므로, 각 토큰의 위치 정보를 임베딩 벡터에 더해줌으로써 모델이 단어의 순서와 위치를 이해할 수 있도록 합니다. |

In [1]:
!mkdir -p ~/work/transformer_chatbot/data/ && cd ~/work/transformer_chatbot/data/
!wget https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv
!pip install sentencepiece

--2025-08-20 08:41:54--  https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv [following]
--2025-08-20 08:41:54--  https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889842 (869K) [text/plain]
Saving to: ‘ChatbotData.csv’

ChatbotData.csv     100%[===================>] 868.99K  3.34MB/s    in 0.3s    

2025-08-20 08:41:55 (3.34 MB/s) - ‘ChatbotData.csv’ saved [889842/889842]



In [2]:
# 1. 라이브러리 설치 및 임포트
!pip install sentencepiece

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import sentencepiece as spm

import csv
import math
import os
import re
import numpy as np

In [3]:
# 2. 데이터 다운로드 및 토크나이저 학습
!wget -nc https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv

def read_raw_data(csv_path):
    pairs = []
    with open(csv_path, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader, None) # 헤더 스킵
        for row in reader:
            if len(row) > 1 and row[0] and row[1]:
                pairs.append((row[0], row[1]))
    return pairs

# 말뭉치 파일 생성
corpus_file = "chatbot_corpus.txt"
raw_pairs = read_raw_data("ChatbotData.csv")
with open(corpus_file, 'w', encoding='utf-8') as f:
    for q, a in raw_pairs:
        f.write(q + "\n")
        f.write(a + "\n")

# SentencePiece BPE 모델 학습
spm.SentencePieceTrainer.Train(
    f'--input={corpus_file} --model_prefix=spm_chatbot --vocab_size=8000 '
    '--model_type=bpe --max_sentence_length=9999 '
    '--pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3'
)

# 토크나이저 로드
sp = spm.SentencePieceProcessor()
sp.Load("spm_chatbot.model")

File ‘ChatbotData.csv’ already there; not retrieving.



True

In [4]:
# 3. GPT-1을 위한 데이터셋 및 위치 인코딩 구현

def preprocess_sentence(sentence: str) -> str:
    sentence = str(sentence).strip()
    sentence = re.sub(r"\s+", " ", sentence)
    sentence = re.sub(r"[\u200b\u200c\u200d\ufeff]", "", sentence)
    sentence = sentence.strip()
    return sentence

class GPT1Dataset(Dataset):
    def __init__(self, pairs, sp, max_length=50):
        super().__init__()
        self.sp = sp
        self.max_length = max_length
        self.data = []

        bos_id = sp.bos_id()
        eos_id = sp.eos_id()
        pad_id = sp.pad_id()
        sep_id = eos_id # 질문과 답변의 구분자로 EOS 토큰 ID를 사용

        #토크나이저를 통해 질문+답변 형태로 바꾸어 시퀀스화 해준다.
        for q_text, a_text in pairs:
            q_text = preprocess_sentence(q_text)
            a_text = preprocess_sentence(a_text)

            q_ids = sp.EncodeAsIds(q_text)
            a_ids = sp.EncodeAsIds(a_text)

            #시퀀스 토큰
            token_ids = [bos_id] + q_ids + [sep_id] + a_ids + [eos_id]

            if len(token_ids) > max_length:
                continue

            padding = [pad_id] * (max_length - len(token_ids))
            input_ids = token_ids + padding
            target_ids = input_ids[1:] + [pad_id]

            self.data.append({"input_ids": input_ids, "target_ids": target_ids})

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        input_ids = torch.tensor(sample["input_ids"], dtype=torch.long)
        target_ids = torch.tensor(sample["target_ids"], dtype=torch.long)
        return input_ids, target_ids

class PositionalEncoding(nn.Module):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(position, d_model)
        pos = torch.arange(0, position, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div_term)
        pe[:, 1::2] = torch.cos(pos * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]

In [5]:
# 4. 데이터셋 구성 확인
pairs = read_raw_data("ChatbotData.csv")
gpt_dataset = GPT1Dataset(pairs, sp, max_length=50)
dataloader = DataLoader(gpt_dataset, batch_size=1, shuffle=False)

sample_input, sample_target = next(iter(dataloader))

print("✅ 입력 데이터가 성공적으로 구성되었습니다.\n")
print("--- 샘플 데이터 확인 ---")
print(f"입력 (Input)  : {sample_input[0].tolist()}")
print(f"타겟 (Target) : {sample_target[0].tolist()}")
print("-" * 20)

decoded_input = sp.decode(sample_input[0].tolist())
decoded_target = sp.decode(sample_target[0].tolist())

print(f"입력 (Decoded)  : {decoded_input}")
print(f"타겟 (Decoded) : {decoded_target}")

✅ 입력 데이터가 성공적으로 구성되었습니다.

--- 샘플 데이터 확인 ---
입력 (Input)  : [2, 5566, 6957, 3207, 7063, 3, 4489, 211, 5936, 6916, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
타겟 (Target) : [5566, 6957, 3207, 7063, 3, 4489, 211, 5936, 6916, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
--------------------
입력 (Decoded)  : 12시 땡! 하루가 또 가네요.
타겟 (Decoded) : 12시 땡! 하루가 또 가네요.


In [6]:
# 5. GPT-1 모델 아키텍처 구현
def scaled_dot_product_attention(query, key, value, mask=None):
    matmul_qk = torch.matmul(query, key.transpose(-1, -2))
    depth = key.size(-1)
    logits = matmul_qk / math.sqrt(depth)
    if mask is not None:
        logits += (mask * -1e9)
    attention_weights = F.softmax(logits, dim=-1)
    output = torch.matmul(attention_weights, value)
    return output, attention_weights

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % num_heads == 0
        self.depth = d_model // num_heads

        self.query_dense = nn.Linear(d_model, d_model)
        self.key_dense = nn.Linear(d_model, d_model)
        self.value_dense = nn.Linear(d_model, d_model)
        self.out_dense = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        query = self.split_heads(self.query_dense(query), batch_size)
        key = self.split_heads(self.key_dense(key), batch_size)
        value = self.split_heads(self.value_dense(value), batch_size)

        scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
        scaled_attention = scaled_attention.permute(0, 2, 1, 3).contiguous()
        concat_attention = scaled_attention.view(batch_size, -1, self.d_model)
        output = self.out_dense(concat_attention)
        return output

def create_look_ahead_mask(x):
    seq_len = x.size(1)
    look_ahead_mask = 1 - torch.tril(torch.ones((seq_len, seq_len), device=x.device))
    padding_mask = (x == sp.pad_id()).float().unsqueeze(1).unsqueeze(2)
    combined_mask = torch.max(look_ahead_mask, padding_mask.squeeze(2))
    return combined_mask.unsqueeze(1)

class GPT1DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ff_dim, dropout=0.1):
        super(GPT1DecoderLayer, self).__init__()
        self.self_mha = MultiHeadAttention(d_model, num_heads)
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout1 = nn.Dropout(dropout)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, d_model)
        )
        self.norm2 = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, look_ahead_mask):
        attn_output = self.self_mha(x, x, x, mask=look_ahead_mask)
        attn_output = self.dropout1(attn_output)
        out1 = self.norm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        out2 = self.norm2(out1 + ffn_output)
        return out2

class GPT1(nn.Module):
    def __init__(self, vocab_size, num_layers, ff_dim, d_model, num_heads, max_len, dropout=0.1):
        super(GPT1, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(max_len, d_model)
        self.dropout = nn.Dropout(dropout)
        self.dec_layers = nn.ModuleList([
            GPT1DecoderLayer(d_model, num_heads, ff_dim, dropout) for _ in range(num_layers)
        ])
        self.final_linear = nn.Linear(d_model, vocab_size)
        self.d_model = d_model

    def forward(self, x):
        look_ahead_mask = create_look_ahead_mask(x)
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        x = self.dropout(x)
        for layer in self.dec_layers:
            x = layer(x, look_ahead_mask)
        logits = self.final_linear(x)
        return logits

In [7]:
# 6. 모델 및 학습 파라미터 설정

# 하이퍼파라미터 (GPT-1 Small과 유사하게 설정)
VOCAB_SIZE = 8000
NUM_LAYERS = 12
D_MODEL = 768
NUM_HEADS = 12
UNITS = 3072 # Feed-Forward 차원 (d_model * 4)
MAX_LEN = 50
DROPOUT = 0.1
EPOCHS = 5 # 예시로 5 에폭만 설정
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 모델 생성
gpt_model = GPT1(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    ff_dim=UNITS,
    max_len=MAX_LEN,
    dropout=DROPOUT
).to(device)

# 손실 함수 및 옵티마이저
loss_function = nn.CrossEntropyLoss(ignore_index=sp.pad_id())
optimizer = optim.Adam(gpt_model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)

# 데이터로더
train_dataloader = DataLoader(gpt_dataset, batch_size=BATCH_SIZE, shuffle=True)

Using device: cpu


In [ ]:
# 7. 학습 루프 정의

def accuracy_function(y_pred, y_true, pad_id=0):
    preds = y_pred.argmax(dim=-1)
    mask = (y_true != pad_id)
    correct = (preds == y_true) & mask
    acc = correct.float().sum() / mask.float().sum()
    return acc

def train_model(model, dataloader, optimizer, loss_fn, num_epochs, device):
    for epoch in range(num_epochs):
        model.train()
        total_loss, total_acc = 0, 0

        for step, (input_ids, target_ids) in enumerate(dataloader):
            input_ids, target_ids = input_ids.to(device), target_ids.to(device)

            optimizer.zero_grad()

            logits = model(input_ids)

            loss = loss_fn(logits.permute(0, 2, 1), target_ids)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_acc += accuracy_function(logits, target_ids, pad_id=sp.pad_id())

            if step % 50 == 0:
                print(f"[Epoch {epoch+1}/{num_epochs}, Step {step}] Loss: {loss.item():.4f}")

        avg_loss = total_loss / len(dataloader)
        avg_acc = total_acc / len(dataloader)
        print(f"\nEpoch {epoch+1} Completed - Avg Loss: {avg_loss:.4f}, Avg Acc: {avg_acc:.4f}\n")

# Pre-train을 위한 학습 시작
train_model(gpt_model, train_dataloader, optimizer, loss_function, EPOCHS, device)

[Epoch 1/5, Step 0] Loss: 9.2061


In [9]:
# 8. 문장 생성(추론) 함수 정의

def sentence_generation(model, sentence, tokenizer, max_gen_len=50, device='cpu'):
    model.eval()

    bos_id = tokenizer.bos_id()
    eos_id = tokenizer.eos_id()
    sep_id = eos_id

    sentence = preprocess_sentence(sentence)
    token_ids = tokenizer.encode(sentence)

    # 모델 입력: [BOS] + Q_ids + [SEP]
    input_ids = [bos_id] + token_ids + [sep_id]

    with torch.no_grad():
        for _ in range(max_gen_len):
            input_tensor = torch.tensor([input_ids], dtype=torch.long, device=device)
            logits = model(input_tensor)
            pred_id = logits.argmax(dim=-1)[0, -1].item()

            if pred_id == eos_id:
                break

            input_ids.append(pred_id)

    start_of_answer = len(token_ids) + 2
    generated_sentence = tokenizer.decode(input_ids[start_of_answer:])

    print("입력 :", sentence)
    print("출력 :", generated_sentence)
    return generated_sentence

# 추론 예시 (실제로는 학습 후에 실행해야 합니다.)
# sentence_generation(gpt_model, "오늘 하루 어땠어?", sp, device=device)